In [4]:
import tensorflow as tf
from tensorflow.keras import layers, Model

def create_wildfire_cnn(input_shape=(64, 64, 3)):
    """
    Creates a CNN model that can be used to predict wildfire spread probability 
    or any similar scalar metric relevant to fire percolation.

    Parameters
    ----------
    input_shape : tuple
        Shape of the input tensor, including channels (default is (64, 64, 3)).

    Returns
    -------
    model : tf.keras.Model
        A compiled Keras model ready for training.
    """
    
    # Define model input
    inputs = tf.keras.Input(shape=input_shape, name="input_layer")
    
    # Convolutional block 1
    x = layers.Conv2D(filters=16, kernel_size=(3, 3), padding='same', activation='relu')(inputs)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    
    # Convolutional block 2
    x = layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    
    # Convolutional block 3
    x = layers.Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    # Flatten the features
    x = layers.Flatten()(x)
    
    # Fully connected layer(s)
    x = layers.Dense(units=128, activation='relu')(x)
    
    # Output layer
    # For a single scalar probability, use a single unit + sigmoid.
    # For a map output, you would use a different shape (e.g., (h*w) or (h, w, 1)) 
    # and possibly no flatten, to keep the spatial resolution.
    outputs = layers.Dense(units=1, activation='sigmoid')(x)
    
    model = Model(inputs=inputs, outputs=outputs, name="WildfireCNN")
    
    # Compile the model
    #   - Binary crossentropy is a common choice if you're predicting a binary outcome
    #     (e.g., will this patch catch fire?). 
    #   - For multi-class or more complex tasks, change as needed.
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# # Example usage:
# if __name__ == "__main__":
#     # Create the CNN
#     model = create_wildfire_cnn(input_shape=(64, 64, 3))
#     model.summary()

#     # Synthetic example data (batch_size=8, 64x64 images, 3 channels)
#     import numpy as np
#     X_dummy = np.random.rand(8, 64, 64, 3).astype('float32')
#     y_dummy = np.random.randint(0, 2, size=(8, 1)).astype('float32')

#     # Train on dummy data (just for demonstration)
#     model.fit(X_dummy, y_dummy, epochs=2, batch_size=4)

Model: "WildfireCNN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 64, 64, 16)     │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 32, 32, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 32, 32, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │       524,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 548,129 (2.09 MB)

 Trainable params: 548,129 (2.09 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/2
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7500 - loss: 0.6984 
Epoch 2/2
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7500 - loss: 0.6124


In [2]:
import rasterio
import numpy as np

def load_fuel_data(geotiff_path, bounding_box=None, out_shape=(64, 64)):
    """
    Load and preprocess fuel/land cover data from a GeoTIFF file.
    - bounding_box: (minX, minY, maxX, maxY) in the raster's CRS
    - out_shape: desired (height, width) for final array
    
    Returns:
    -------
    fuel_array : np.ndarray of shape (64, 64)
        Resampled 2D array representing fuel or land cover.
    """
    with rasterio.open(geotiff_path) as src:
        # If bounding_box is specified, read only that window
        if bounding_box:
            # Convert bounding box to rasterio window
            # This step requires coordinate transformations as needed
            window = rasterio.windows.from_bounds(*bounding_box, transform=src.transform)
            data = src.read(1, window=window, out_shape=out_shape, resampling=rasterio.enums.Resampling.bilinear)
        else:
            # Read entire raster and then resample
            data = src.read(1)
            
            # Resample to out_shape if needed
            # Create transform for desired out_shape
            data = np.array(
                rasterio.resize.resize(data, out_shape, resampling=rasterio.enums.Resampling.bilinear),
                dtype=np.float32
            )
            
    # Normalize or transform the fuel values as needed
    # e.g., min-max normalization or some domain-specific re-scaling
    data = (data - data.min()) / (data.max() - data.min() + 1e-6)
    return data

In [5]:
def load_historic_temperature_data(geotiff_path, bounding_box=None, out_shape=(64, 64)):
    """
    Load and preprocess historic temperature data from a GeoTIFF file.
    Similar structure to load_fuel_data.
    """
    import rasterio
    
    with rasterio.open(geotiff_path) as src:
        if bounding_box:
            window = rasterio.windows.from_bounds(*bounding_box, transform=src.transform)
            temp_data = src.read(1, window=window, out_shape=out_shape, resampling=rasterio.enums.Resampling.bilinear)
        else:
            temp_data = src.read(1)
            temp_data = np.array(
                rasterio.resize.resize(temp_data, out_shape, resampling=rasterio.enums.Resampling.bilinear),
                dtype=np.float32
            )

    # Example: Convert from Kelvin to Celsius (if needed), then scale
    # Here we assume the data is already in a known range
    temp_data = (temp_data - temp_data.min()) / (temp_data.max() - temp_data.min() + 1e-6)
    return temp_data

In [1]:
def combine_datasets(fuel_array, temp_array):
    """
    Combine 2D arrays of fuel and temperature into a single 3D tensor (64, 64, 2).
    """
    # Ensure both are (64, 64)
    assert fuel_array.shape == temp_array.shape, "Fuel and temperature arrays must match shape."
    
    # Stack along last axis
    combined = np.stack([fuel_array, temp_array], axis=-1)  # shape = (64, 64, 2)
    return combined

In [ ]:

# Example usage:
# Provide paths to your actual USGS GeoTIFFs or other data source
fuel_geotiff = "path/to/usgs_landcover_fuel.tif"
temp_geotiff = "path/to/historic_temp_data.tif"

# Optional bounding box (minX, minY, maxX, maxY) in coordinate system of the TIFF
bbox = (-120.5, 35.0, -120.0, 35.5)  # Dummy bounding box

# 1) Load data
fuel_array = load_fuel_data(fuel_geotiff, bounding_box=bbox, out_shape=(64, 64))
temp_array = load_historic_temperature_data(temp_geotiff, bounding_box=bbox, out_shape=(64, 64))

# 2) Combine channels
input_data = combine_datasets(fuel_array, temp_array)  # shape: (64, 64, 2)

# Expand dims to create a "batch" of size 1
input_data = np.expand_dims(input_data, axis=0)  # shape: (1, 64, 64, 2)

# 3) Create the model
model = create_wildfire_cnn(input_shape=(64, 64, 2))
model.summary()

# 4) Inference or training example
#    For training, you'd need labeled data (e.g., known burnt/unburnt outcomes).
#    Here we just do a forward pass for demonstration.
prediction = model.predict(input_data)
print("Model output:", prediction)